In [2]:
import mlcTest as mt
import pandas as pd
import tensorflow as tf

{'signal': [0.18531812727451324, 0.9329000115394592]}


In [156]:
data = pd.read_csv('test_result.csv')
predicted_cons = []
for i in range(len(data)):
    if data.prediction[i] in [j for j in range(0,4)]:
        predicted_cons.append(1)
    else:
        predicted_cons.append(2)
test = data.assign(predicted_cons=predicted_cons)

In [48]:
data

,real,imag,cons,block,label,interference,buffer,label_real,label_imag,prediction
0,0.120629,0.117264,2,1,10,1,(0.31622776601683794+0.31622776601683794j),0.316228,0.316228,10
1,-0.038891,0.152451,2,1,9,1,(-0.31622776601683794+0.31622776601683794j),-0.316228,0.316228,9
2,0.481826,-0.876530,2,1,19,1,(0.9486832980505138-0.9486832980505138j),0.948683,-0.948683,19
3,0.533483,-0.142285,2,1,15,1,(0.9486832980505138-0.31622776601683794j),0.948683,-0.316228,15
4,0.421070,0.472924,2,1,10,1,(0.31622776601683794+0.31622776601683794j),0.316228,0.316228,6
...,...,...,...,...,...,...,...,...,...,...
9595,-0.167899,-0.299525,2,993,13,0,(-0.31622776601683794-0.31622776601683794j),-0.316228,-0.316228,13
9596,-0.568121,0.242732,2,993,8,0,(-0.9486832980505138+0.31622776601683794j),-0.948683,0.316228,8
9597,-0.157070,-0.660922,2,993,17,0,(-0.31622776601683794-0.9486832980505138j),-0.316228,-0.948683,17
9598,-0.210914,-0.207288,2,993,13,0,(-0.31622776601683794-0.31622776601683794j),-0.316228,-0.316228,13


# 1 
- P(Constellation Error | Constellation = 4QAM)
- P(Constellation Error | Constellation = 16QAM)

In [43]:
real_label = data.label
pred_label = data.prediction

In [44]:
cf = tf.math.confusion_matrix(real_label, pred_label).numpy()
cf = pd.DataFrame(cf)

In [247]:
cons_4 = test[test.cons==1]
cons_16 = test[test.cons==2]
total_cons_error = cons_4_error + cons_16_error
cons_4_error_rate = cons_4_error / total_cons_error
cons_16_error_rate = cons_16_error / total_cons_error

In [249]:
print("CONS_ERROR|4QAM  IS: ", cons_4_error_rate)
print("CONS_ERROR|16QAM IS: ", cons_16_error_rate)

CONS_ERROR|4QAM  IS:  0.6870540265035678
CONS_ERROR|TOTAL IS:  0.3129459734964322


# 2
- P(Constellation Error | Interference)
- P(Constellation Error | No Interference)

In [181]:
data_with_ins = data[data.interference==1]
ins_label = data_with_ins.label
ins_prediction = data_with_ins.prediction
cons_error = sum(test.cons!=test.predicted_cons)

In [50]:
cf_ins = tf.math.confusion_matrix(cons_label, cons_prediction).numpy()
cf_ins = pd.DataFrame(cf_cons)

In [182]:
interference_data = test[data.interference==1]
interference_cons_error = sum(interference_data.cons!=interference_data.predicted_cons)
n_interference_data = test[data.interference==0]
n_interference_cons_error = sum(n_interference_data.cons!=n_interference_data.predicted_cons)
interference_cons_error_rate = interference_cons_error / cons_error
n_interference_cons_error_rate = n_interference_cons_error / cons_error

In [183]:
print("CONS_ERROR|Interference is: ", interference_cons_error_rate)
print("CONS_ERROR|No Interference is: ", n_interference_cons_error_rate)

CONS_ERROR|Interference is:  0.8868501529051988
CONS_ERROR|No Interference is:  0.11314984709480122


# 3
- P(Symbol Error | Constellation = 4QAM AND correct constellation decision)
- P(Symbol Error | Constellation = 16QAM AND correct constellation decision)

In [184]:
cons_4_data = test[test.cons==1]
cons_16_data = test[test.cons==2]

In [185]:
cons_4_symbol_error = sum(cons_4_data.label!=cons_4_data.prediction)
cons_16_symbol_error = sum(cons_16_data.label!=cons_16_data.prediction)

In [197]:
cons_4_symbol_error_cons_correct = cons_4_data[cons_4_data.label!=cons_4_data.prediction]
cons_4_symbol_error_cons_correct = cons_4_symbol_error_cons_correct[cons_4_symbol_error_cons_correct.cons ==
                                                                   cons_4_symbol_error_cons_correct.predicted_cons]
cons_16_symbol_error_cons_correct = cons_16_data[cons_16_data.label!=cons_16_data.prediction]
cons_16_symbol_error_cons_correct = cons_16_symbol_error_cons_correct[cons_16_symbol_error_cons_correct.cons ==
                                                                   cons_16_symbol_error_cons_correct.predicted_cons]

In [198]:
p_cons_4_symbolError_consCorrect_rate = len(cons_4_symbol_error_cons_correct) / total_error
p_cons_16_symbolError_consCorrect_rate = len(cons_16_symbol_error_cons_correct) / total_error

In [199]:
print("P(Symbol Error | Constellation = 4QAM AND correct constellation decision) = ", 
     p_cons_4_symbolError_consCorrect_rate)
print("P(Symbol Error | Constellation = 16QAM AND correct constellation decision) = ",
     p_cons_16_symbolError_consCorrect_rate)

P(Symbol Error | Constellation = 4QAM AND correct constellation decision) =  0.0
P(Symbol Error | Constellation = 16QAM AND correct constellation decision) =  0.3795066413662239


# 4
- P(Symbol Error | Interference AND correct constellation decision)
- P(Symbol Error | No Interference AND correct constellation decision)

In [200]:
interference_data = test[test.interference==1]

In [201]:
interference_correct_cons = interference_data[interference_data.cons==interference_data.predicted_cons]

In [203]:
cons_correct_symbol_error = interference_correct_cons[interference_correct_cons.label!=
                                                      interference_correct_cons.prediction]

In [205]:
cons_correct_symbol_error_rate = len(cons_correct_symbol_error)/total_error

In [210]:
n_interference_data = test[test.interference==0]
n_interference_symbol_error = n_interference_data[n_interference_data.label!=
                                                n_interference_data.prediction]
n_cons_correct_symbol_error = n_interference_symbol_error[n_interference_symbol_error.cons==
                                                      n_interference_symbol_error.predicted_cons]
n_cons_correct_symbol_error_rate = len(n_cons_correct_symbol_error)/total_error

In [211]:
print("P(Symbol Error | Interference AND correct constellation decision) = ", 
      cons_correct_symbol_error_rate)
print("P(Symbol Error | No Interference AND correct constellation decision) = ", 
     n_cons_correct_symbol_error_rate)

P(Symbol Error | Interference AND correct constellation decision) =  0.3788741302972802
P(Symbol Error | No Interference AND correct constellation decision) =  0.0006325110689437065


# 5
- P(Interference | Constellation Error)
- P(Interference | No Constellation Error)

In [167]:
interference_data = test[test.interference==1]
interference_cons_error = sum(interference_data.cons!=interference_data.predicted_cons)

In [212]:
interference_cons_error_rate = interference_cons_error/len(interference_data)

In [214]:
interference_cons_n_error = sum(interference_data.cons==interference_data.predicted_cons)

In [215]:
interference_cons_n_error_rate = interference_cons_n_error/len(interference_data)

In [216]:
print("P(Interference | Constellation Error) = ", interference_cons_error_rate)
print("P(Interference | No Constellation Error) = ", interference_cons_n_error_rate)

P(Interference | Constellation Error) =  0.12253521126760564
P(Interference | No Constellation Error) =  0.8774647887323944


In [11]:
result = "./result/method"
test_result = []
for i in range(1,11):
    i = str(i)
    data = pd.read_csv(result+i+".csv")
    test_result.append(data)
test = pd.concat(test_result, ignore_index=True)

In [13]:
test.index += 1

In [14]:
test

,Unnamed: 0,gen_loss,gen_relative_loss,signal_loss,signal_relative_loss,noise_loss,noise_relative_loss
1,0,0.018536,3.903176,0.191420,54.468190,0.057665,98.61353
2,0,0.020388,234.106350,0.167512,-16.027079,0.058447,453.40950
3,0,0.017184,1.485669,0.159658,9.362474,0.058028,397.06805
4,0,0.017878,-10.840482,0.160939,-56.187180,0.058335,-512.63350
5,0,0.017923,8.077351,0.162093,-28.411873,0.058148,417.12973
6,0,0.017491,2.484557,0.160338,34.378640,0.057734,565.52010
7,0,0.018586,-10.530231,0.161917,-20.161768,0.058923,-249.28694
8,0,0.019018,1.679274,0.165585,16.856802,0.058480,2276.33130
9,0,0.018424,1.647986,0.159293,17.752644,0.058828,375.52936
10,0,0.020115,12.817821,0.160217,-78.900410,0.057928,-436.68677
